In [1]:
import cv2
import numpy as np
import face_recognition
import cvlib as cv


cap = cv2.VideoCapture(0) #if you are using webcam
cap.set(3, 480) #set width of the frame
cap.set(4, 640) #set height of the frame

obama_image = face_recognition.load_image_file("obama.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]

biden_image = face_recognition.load_image_file("biden.jpg")
biden_face_encoding = face_recognition.face_encodings(biden_image)[0]

nacho_image = face_recognition.load_image_file("nacho.jpg")
nacho_face_encoding = face_recognition.face_encodings(nacho_image)[0]

# Create arrays of known face encodings and their names
known_face_encodings = [
    obama_face_encoding,
    biden_face_encoding,
    nacho_face_encoding
]
known_face_names = [
    "Barack Obama",
    "Joe Biden",
    "Ignacio Gonzalez"
]

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []


MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
gender_list = ['Male', 'Female']

def load_caffe_models():
    age_net = cv2.dnn.readNetFromCaffe('C:/Users/Nacho/deploy_age.prototxt', 'C:/Users/Nacho/age_net.caffemodel')
    gender_net = cv2.dnn.readNetFromCaffe('C:/Users/Nacho/deploy_gender.prototxt', 'C:/Users/Nacho/gender_net.caffemodel')
    return(age_net, gender_net)

def video_detector(age_net, gender_set):
    font = cv2.FONT_HERSHEY_SIMPLEX
    process_this_frame = True
    while True:
        ret, image = cap.read()
        face_cascade = cv2.CascadeClassifier('C:/Users/Nacho/Paso a paso Reconocimiento Facial/detectors/haarcascade_frontalface_alt.xml')
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray,1.1,5)
        
        
        small_image = cv2.resize(image,(0,0),fx=0.25,fy=0.25)
        rgb_small_frame = small_image[:,:,::-1]
        
        if(len(faces)>0):
            print("Found {} faces.".format(str(len(faces))))
        
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

            face_names = []
            for face_encoding in face_encodings:
                # See if the face is a match for the known face(s)
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                # # If a match was found in known_face_encodings, just use the first one.
                # if True in matches:
                #     first_match_index = matches.index(True)
                #     name = known_face_names[first_match_index]

                # Or instead, use the known face with the smallest distance to the new face
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = np.argmin(face_distances)
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]

                face_names.append(name)

        process_this_frame = not process_this_frame
        for (x,y,w,h) in faces:
            cv2.rectangle(image,(x,y),(x+w,y+h),(255,255,0),2)

            #Get face
            face_img = image[y:y+h,h:h+w].copy()
            blob = cv2.dnn.blobFromImage(face_img,1,(227,227),MODEL_MEAN_VALUES,swapRB=False)

        #Predict Gender
            gender_net.setInput(blob)
            gender_preds = gender_net.forward()
            gender = gender_list[gender_preds[0].argmax()]
            print("Gender: " + gender)

        #Predict Age
            age_net.setInput(blob)
            age_preds =age_net.forward()
            age = age_list[age_preds[0].argmax()]
            print("Age Range: " + age)

            overlay_text = "%s %s %s" % (name,gender,age)
            cv2.putText(image,overlay_text,(x,y),font,1,(255,255,255),2,cv2.LINE_AA)
            cv2.imshow('frame',image)
            
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        
if __name__ == "__main__":
    age_net, gender_net = load_caffe_models()

    video_detector(age_net,gender_net)

Found 1 faces.
Gender: Female
Age Range: (8, 12)
Found 1 faces.
Gender: Female
Age Range: (4, 6)
Found 1 faces.
Gender: Female
Age Range: (15, 20)
Found 1 faces.
Gender: Female
Age Range: (8, 12)
Found 1 faces.
Gender: Female
Age Range: (8, 12)
Found 1 faces.
Gender: Female
Age Range: (15, 20)
Found 1 faces.
Gender: Female
Age Range: (0, 2)
Found 1 faces.
Gender: Female
Age Range: (0, 2)
Found 1 faces.
Gender: Female
Age Range: (25, 32)
Found 1 faces.
Gender: Female
Age Range: (25, 32)
Found 1 faces.
Gender: Female
Age Range: (25, 32)
Found 1 faces.
Gender: Female
Age Range: (25, 32)
Found 1 faces.
Gender: Female
Age Range: (0, 2)
Found 1 faces.
Gender: Female
Age Range: (25, 32)
Found 1 faces.
Gender: Female
Age Range: (25, 32)
Found 1 faces.
Gender: Female
Age Range: (25, 32)
Found 1 faces.
Gender: Female
Age Range: (25, 32)
Found 1 faces.
Gender: Female
Age Range: (25, 32)
Found 1 faces.
Gender: Female
Age Range: (25, 32)
Found 1 faces.
Gender: Female
Age Range: (25, 32)
Found 1 fac